In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
con_dev = ['MLCim', 'MLFaren', 'MLHabibi']

In [3]:
base_dir = '/content/drive/MyDrive/'+con_dev[2]+'/ChaMi-ML/Train'
%cd '$base_dir'

/content/drive/MyDrive/MLHabibi/ChaMi-ML/Train


In [4]:
# import all package
import data, model, train

FileExistsError: ignored